# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 22 2022 12:29PM,240191,21,617959-1,"NBTY Global, Inc.",Released
1,Jun 22 2022 12:27PM,240190,10,Enova-10190,Emerginnova,Released
2,Jun 22 2022 12:27PM,240190,10,Enova-10191,Emerginnova,Released
3,Jun 22 2022 12:17PM,240189,19,ADV80003491,"AdvaGen Pharma, Ltd",Released
4,Jun 22 2022 12:16PM,240188,10,MSP211544,"Methapharm, Inc.",Released
5,Jun 22 2022 11:55AM,240187,10,SO94292,"Senseonics, Incorporated",Released
6,Jun 22 2022 11:55AM,240187,10,SO94293,"Senseonics, Incorporated",Released
7,Jun 22 2022 11:55AM,240187,10,SO94294,"Senseonics, Incorporated",Released
8,Jun 22 2022 11:55AM,240187,10,SO94295,"Senseonics, Incorporated",Released
9,Jun 22 2022 11:55AM,240187,10,SO94296,"Senseonics, Incorporated",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,240187,Released,6
32,240188,Released,1
33,240189,Released,1
34,240190,Released,2
35,240191,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
240187,NaN,6.0
240188,NaN,1.0
240189,NaN,1.0
240190,NaN,2.0
240191,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
240089,0.0,1.0
240092,1.0,0.0
240102,1.0,1.0
240124,0.0,1.0
240133,17.0,21.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
240089,0,1
240092,1,0
240102,1,1
240124,0,1
240133,17,21


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,240089,0,1
1,240092,1,0
2,240102,1,1
3,240124,0,1
4,240133,17,21


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,240089,,1
1,240092,1,
2,240102,1,1
3,240124,,1
4,240133,17,21


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 22 2022 12:29PM,240191,21,"NBTY Global, Inc."
1,Jun 22 2022 12:27PM,240190,10,Emerginnova
3,Jun 22 2022 12:17PM,240189,19,"AdvaGen Pharma, Ltd"
4,Jun 22 2022 12:16PM,240188,10,"Methapharm, Inc."
5,Jun 22 2022 11:55AM,240187,10,"Senseonics, Incorporated"
11,Jun 22 2022 11:35AM,240185,10,Emerginnova
14,Jun 22 2022 11:29AM,240184,21,"NBTY Global, Inc."
15,Jun 22 2022 11:22AM,240180,10,ISDIN Corporation
47,Jun 22 2022 11:19AM,240179,10,ISDIN Corporation
55,Jun 22 2022 10:58AM,240177,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 22 2022 12:29PM,240191,21,"NBTY Global, Inc.",,1
1,Jun 22 2022 12:27PM,240190,10,Emerginnova,,2
2,Jun 22 2022 12:17PM,240189,19,"AdvaGen Pharma, Ltd",,1
3,Jun 22 2022 12:16PM,240188,10,"Methapharm, Inc.",,1
4,Jun 22 2022 11:55AM,240187,10,"Senseonics, Incorporated",,6
5,Jun 22 2022 11:35AM,240185,10,Emerginnova,,3
6,Jun 22 2022 11:29AM,240184,21,"NBTY Global, Inc.",,1
7,Jun 22 2022 11:22AM,240180,10,ISDIN Corporation,,32
8,Jun 22 2022 11:19AM,240179,10,ISDIN Corporation,1,7
9,Jun 22 2022 10:58AM,240177,20,"ACI Healthcare USA, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 22 2022 12:29PM,240191,21,"NBTY Global, Inc.",1,
1,Jun 22 2022 12:27PM,240190,10,Emerginnova,2,
2,Jun 22 2022 12:17PM,240189,19,"AdvaGen Pharma, Ltd",1,
3,Jun 22 2022 12:16PM,240188,10,"Methapharm, Inc.",1,
4,Jun 22 2022 11:55AM,240187,10,"Senseonics, Incorporated",6,
5,Jun 22 2022 11:35AM,240185,10,Emerginnova,3,
6,Jun 22 2022 11:29AM,240184,21,"NBTY Global, Inc.",1,
7,Jun 22 2022 11:22AM,240180,10,ISDIN Corporation,32,
8,Jun 22 2022 11:19AM,240179,10,ISDIN Corporation,7,1
9,Jun 22 2022 10:58AM,240177,20,"ACI Healthcare USA, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 22 2022 12:29PM,240191,21,"NBTY Global, Inc.",1,
1,Jun 22 2022 12:27PM,240190,10,Emerginnova,2,
2,Jun 22 2022 12:17PM,240189,19,"AdvaGen Pharma, Ltd",1,
3,Jun 22 2022 12:16PM,240188,10,"Methapharm, Inc.",1,
4,Jun 22 2022 11:55AM,240187,10,"Senseonics, Incorporated",6,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 22 2022 12:29PM,240191,21,"NBTY Global, Inc.",1.0,NaN
1,Jun 22 2022 12:27PM,240190,10,Emerginnova,2.0,NaN
2,Jun 22 2022 12:17PM,240189,19,"AdvaGen Pharma, Ltd",1.0,NaN
3,Jun 22 2022 12:16PM,240188,10,"Methapharm, Inc.",1.0,NaN
4,Jun 22 2022 11:55AM,240187,10,"Senseonics, Incorporated",6.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 22 2022 12:29PM,240191,21,"NBTY Global, Inc.",1.0,0.0
1,Jun 22 2022 12:27PM,240190,10,Emerginnova,2.0,0.0
2,Jun 22 2022 12:17PM,240189,19,"AdvaGen Pharma, Ltd",1.0,0.0
3,Jun 22 2022 12:16PM,240188,10,"Methapharm, Inc.",1.0,0.0
4,Jun 22 2022 11:55AM,240187,10,"Senseonics, Incorporated",6.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2882071,79.0,28.0
102,240089,1.0,0.0
15,1200769,60.0,40.0
19,1440815,5.0,2.0
20,480335,2.0,0.0
21,480375,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2882071,79.0,28.0
1,102,240089,1.0,0.0
2,15,1200769,60.0,40.0
3,19,1440815,5.0,2.0
4,20,480335,2.0,0.0
5,21,480375,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,79.0,28.0
1,102,1.0,0.0
2,15,60.0,40.0
3,19,5.0,2.0
4,20,2.0,0.0
5,21,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,79.0
1,102,Released,1.0
2,15,Released,60.0
3,19,Released,5.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,19,20,21
Status,,,,,,
Executing,28.0,0.0,40.0,2.0,0.0,0.0
Released,79.0,1.0,60.0,5.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,19,20,21
0,Executing,28.0,0.0,40.0,2.0,0.0,0.0
1,Released,79.0,1.0,60.0,5.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,19,20,21
0,Executing,28.0,0.0,40.0,2.0,0.0,0.0
1,Released,79.0,1.0,60.0,5.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()